# Primera Red Neuronal - Perceptrón multicapa

## Clasificación de dígitos escritos a mano

![mnist](mnist.png)

Cada imagen del conjunto de datos MNIST contiene 28x28 pixeles. Vamos a aplanar estos pixeles en un vector de 784 unidades, que usaremos como entradas de la red. Las salidas corresponden a 10 casos posibles, uno por cada dígito.

## Configuración

Instalación de paquetes

In [1]:
!pip install keras tensorflow numpy mnist

Estas anteriores hacen parte de los paquetes más utilizadas para crear redes neuronales

In [2]:
import numpy as np
import mnist
import keras

Using TensorFlow backend.


In [3]:
# La primera vez que se corre esto puede ser lento
# porque mnist necesita descargar los datos
train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

(60000, 28, 28)
(60000,)


## Preparar los datos

Vamos a aplanar las imágenes para ingresarlas a la red neuronal. También, vamos a normalizar los valores de los pixeles de [0, 255] a [-0.5, 0.5] para hacer más fácil el entrenamiento (usar valores centrados más pequeños suele ser mejor).

In [4]:
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalizar imágenes
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Aplanar imágenes
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 784)
(10000, 784)


Todo listo para la red neuronal

## Construir el modelo

Cada modelo en Keras es construido utilizando la clase 'Sequential' que representa una pila de capas lineales, o por otro lado, utilizar 'Model' que tiene más opciones de personalización.

In [5]:
from keras.models import Sequential
from keras.layers import Dense

# Inicializar
model = Sequential([
  # Capas...
])

El constructor 'Sequential' crea un array de capas de Keras, como estamos construyendo una red de propagación hacia adelante (feedforward), solo necesitamos la capa 'Dense', que conecta las capas.

In [6]:
# Trabajo en progreso
model = Sequential([
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

Las primeras dos capas utilizan 64 neuronas y la función de activación ReLU. La última capa utiliza Softmax con 10 nodos, uno por cada clase.

Por último es necesario especificarle a Keras la forma de los datos de entrada

In [7]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

Keras infiere la forma de las capas posteriores

## Compilar el modelo

Antes de empezar a entrenar es necesario especificar 3 factores clave para el entrenamiento:

- El **optimizador**: Un buen optimizador por defecto es el 'Adam gradient-based'.

- La **función de pérdida**: Como estamos utilizando en la capa de salida Softmax, empleamos la función de entropía cruzada categórica. Keras distingue entre los casos binarios y categóricos (este último el que corresponde a este ejemplo).

- Una lista de **métricas**: Como este es un problema de clasificación, le pediremos a Keras que nos arroje la métrica de precisión.

In [8]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

## Entrenar el modelo

Realizaremos el ajuste del modelo especificando algunos parámetros; en realidad hay muchos pero vamos a suministrar manualmente algunos:

- Los **datos de entrenamiento**: (imágenes y etiquetas) comúnmente conocidos como X y Y, respectivamente.

- El **número de épocas**: iteraciones a realizar sobre el conjunto de datos.

- El **tamaño del lote**: Número de muestras por cada actualización del gradiente.

model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
)

Esto todavía no funciona, Keras espera que las etiquetas sean 10 vectores. Como hay 10 neuronas en la capa de salida, estamos hasta ahora suministrando un entero para cada clase.

Por fortuna, Keras contiene un método para convertir un array de enteros, en un array de vectores 'one - hot': por ejemplo, convertir el número 2 en [0,0,1,0,0,0,0,0,0,0]

In [9]:
from keras.utils import to_categorical
# Entrenar el modelo
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=5,
  batch_size=32,
)

Epoch 1/5
60000/60000 [==============================] - 13s 209us/step - loss: 0.3448 - accuracy: 0.8963
Epoch 2/5
60000/60000 [==============================] - 10s 174us/step - loss: 0.1805 - accuracy: 0.9449
Epoch 3/5
60000/60000 [==============================] - 11s 185us/step - loss: 0.1386 - accuracy: 0.9574
Epoch 4/5
60000/60000 [==============================] - 10s 166us/step - loss: 0.1160 - accuracy: 0.9643
Epoch 5/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.0991 - accuracy: 0.9692


Alcanzamos una precisión de 96.9% con tan solo 5 épocas. Esto no nos dice mucho porque el modelo podría estarse sobreajustando.

## Probar el modelo

Afortunadamente en Keras es bastante sencillo:

In [10]:
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

10000/10000 [==============================] - 1s 83us/step


[0.13791403009481729, 0.9562000036239624]

## Utilizar el modelo

Ahora que tenemos un modelo entrenado, podemos guardarlo para utilizarlo después

In [11]:
model.save_weights('model.h5')

Con el siguiente código reducido tenemos todo el modelo anterior

In [12]:
from keras.models import Sequential
from keras.layers import Dense

# Construir el modelo
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

# Cargar los pesos guardados
model.load_weights('model.h5')

Usar el modelo entrenado para hacer predicciones es fácil, pasamos un array de entradas para predecir con la función 'predict()' y esta retorna un array de salidas. Hay que tener en cuenta que la salidas de softmax son 10 probabilidades, así que utilizaremos 'np.argmax()' para convetir eso en dígitos.

In [13]:
# Predecir sobre las primeras 5 imágenes
predictions = model.predict(test_images[:5])

# Imprimir las predicciones
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Contrastar las predicciones contra las verdaderas observaciones
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[7 2 1 0 4]


## Cuestiones adicionales

Hasta ahora todo ha sido una breve introducción, hay numerosos procedimientos que se pueden realizar para mejorar una red

### Ajustar los hiperparámetros

Un buen hiperparámetro con el que se puede empezar es la **tasa de aprendizaje** del optimizador Adam. ¿Qué sucede si se incrementa o disminuye?

In [14]:
from keras.optimizers import Adam

model.compile(
  optimizer=Adam(lr=0.005),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

Modifiquemos el **tamaño del lote y el número de épocas**

In [15]:
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=10,
  batch_size=64,
)

Epoch 1/10
60000/60000 [==============================] - 6s 98us/step - loss: 0.1808 - accuracy: 0.9449
Epoch 2/10
60000/60000 [==============================] - 5s 86us/step - loss: 0.1466 - accuracy: 0.9548
Epoch 3/10
60000/60000 [==============================] - 5s 86us/step - loss: 0.1336 - accuracy: 0.9592
Epoch 4/10
60000/60000 [==============================] - 6s 93us/step - loss: 0.1254 - accuracy: 0.9622 0s - loss: 0.1
Epoch 5/10
60000/60000 [==============================] - 5s 85us/step - loss: 0.1240 - accuracy: 0.9634
Epoch 6/10
60000/60000 [==============================] - 5s 89us/step - loss: 0.1126 - accuracy: 0.9663
Epoch 7/10
60000/60000 [==============================] - 5s 84us/step - loss: 0.1100 - accuracy: 0.9675
Epoch 8/10
60000/60000 [==============================] - 6s 93us/step - loss: 0.1033 - accuracy: 0.9696
Epoch 9/10
60000/60000 [==============================] - 5s 89us/step - loss: 0.1016 - accuracy: 0.9698
Epoch 10/10
60000/60000 [===============

### Profundidad de la red

¿Qué sucede si agregamos más capas?

In [16]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

### Otra función de activación

Utilizar la función Sigmoide en lugar de la ReLU

In [17]:
model = Sequential([
  Dense(64, activation='sigmoid', input_shape=(784,)),
  Dense(64, activation='sigmoid'),
  Dense(10, activation='softmax'),
])

### Dropout

Capas 'dropout' que sirven para evitar el sobreajuste

In [18]:
from keras.layers import Dense, Dropout

model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dropout(0.5),
  Dense(64, activation='relu'),
  Dropout(0.5),
  Dense(10, activation='softmax'),
])

### Validación

Keras permite evaluar el modelo al final de cada época y reportar la pérdida y cualquier métrica que solicitemos. Esto nos deja monitorear el progreso del modelo durante el entrenamiento, que puede ser útil para identificar sobreajuste.

In [19]:
model.compile(
  optimizer=Adam(lr=0.005),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [20]:
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=5,
  batch_size=32,
  validation_data=(test_images, to_categorical(test_labels))
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 168us/step - loss: 1.4744 - accuracy: 0.4594 - val_loss: 1.0711 - val_accuracy: 0.7124
Epoch 2/5
60000/60000 [==============================] - 8s 138us/step - loss: 1.3713 - accuracy: 0.5088 - val_loss: 0.7769 - val_accuracy: 0.8141
Epoch 3/5
60000/60000 [==============================] - 7s 116us/step - loss: 1.3152 - accuracy: 0.5496 - val_loss: 0.8566 - val_accuracy: 0.7622
Epoch 4/5
60000/60000 [==============================] - 7s 115us/step - loss: 1.2810 - accuracy: 0.5688 - val_loss: 0.7285 - val_accuracy: 0.8191
Epoch 5/5
60000/60000 [==============================] - 9s 147us/step - loss: 1.2841 - accuracy: 0.5663 - val_loss: 0.8936 - val_accuracy: 0.7367


## Conclusión

Abordamos las cuestiones principales para construir una primera red neuronal para problemas de clasificación. El siguiente paso sería aprender sobre redes neuronales convolucionadas.